# Capstone Final Project - The Battle of the Neighborhoods - Exploring Bogota

## Description of the data that we will be using to solve the problem or execute the idea.

In this notebook we are going to explore Bogota´s top venues.

Explore the capital of Colombia is 

For this analysis I choose 2 sources:
1. Bogota city is divided in 20 districts: 19 urban and one rural. Many of them are the oldest towns in Colombia. Each distric is conformed by a group of "UPZ" (Unidades de Planeamiento Zonal) and each UPZ is subdivided in neighborhoods. In this analysis we will explore the districts, to get the geographical coordinates of the localidades we download the csv file of this site:
    
    Link: https://bogota-laburbano.opendatasoft.com/explore/dataset/georeferencia-puntual-por-localidad/table/
    
    
2. Using foursquare API to get the data about venues in Bogota.

With this data we can explore all the venues in each district and find out the top venues for recommendations to tourist.
   

### Import libraries

In [1]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_b3db8fa85a8946ad834caaa4f83f4eba = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='0C8vzYa-FmPgDc17RnZ9bqodBMVxX1HyNMHpvc7d1w6q',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

print("Libraries imported")

Libraries imported


### Dowload and read the data

In [13]:
body = client_b3db8fa85a8946ad834caaa4f83f4eba.get_object(Bucket='capstoneprojectcourse-donotdelete-pr-qxydsseiqruq4s',Key='localidades_bta.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
    
bogota_data = pd.read_csv(body)
bogota_data.head()

,LOCALIDAD,LONGITUD,LATITUD
0,CHAPINERO,-74.0467,4.6569
1,TUNJUELITO,-74.1407,4.5875
2,ANTONIO NARINO,-74.1009,4.5486
3,BARRIOS UNIDOS,-74.0840,4.6664
4,ENGATIVA,-74.1072,4.7071


In [14]:
bogota_data.rename(columns={"LOCALIDAD":"District","LONGITUD":"Longitude","LATITUD":"Latitude"}, inplace=True)
bogota_data = bogota_data[["District","Latitude","Longitude"]]
bogota_data.head()

,District,Latitude,Longitude
0,CHAPINERO,4.6569,-74.0467
1,TUNJUELITO,4.5875,-74.1407
2,ANTONIO NARINO,4.5486,-74.1009
3,BARRIOS UNIDOS,4.6664,-74.0840
4,ENGATIVA,4.7071,-74.1072


In [8]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

### Get the geograpical coordinates of Bogota

In [9]:
address = 'Bogota, CO'
geolocator = Nominatim(user_agent = "bo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Bogota are {}, {}".format(latitude, longitude))

The geograpical coordinate of Bogota are 4.59808, -74.0760439


### Visualize the data

Create a map of Bogota with districts superimposed on top.

In [10]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

In [15]:
map_bogota = folium.Map(location=[latitude, longitude], zoom_start=11)

#add markers to map
for lat, lng, localidad in zip(bogota_data["Latitude"], bogota_data["Longitude"], bogota_data["District"]):
    label = '{}'.format(localidad)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False).add_to(map_bogota)

map_bogota